In [ ]:
import tensorflow as tf
import numpy as np
import time
import os

In [2]:
!python prepare_data2.py shakespeare.txt skp \\n\\n+ -m 200

2020-06-02 06:42:08.165812: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 7222 sequences...
Longest sequence is 3082 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 200 characters...
5666 sequences remaining.
Longest remaining sequence has length 199.
Removing length-0 sequences...
5666 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
S

In [3]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'\n': 3, '-': 4, ':': 5, '?': 6, 'S': 7, 'I': 8, 'i': 9, 'Z': 10, 'b': 11, 'n': 12, 'P': 13, 'V': 14, 'C': 15, 'q': 16, '&': 17, 'Q': 18, 'd': 19, 'R': 20, 'o': 21, '$': 22, 'f': 23, 'J': 24, 'L': 25, 'j': 26, 'E': 27, 'h': 28, 'p': 29, 'm': 30, '3': 31, 'l': 32, 'K': 33, 'W': 34, 'H': 35, ';': 36, 'D': 37, 's': 38, 'G': 39, 'N': 40, 'e': 41, 'B': 42, 'r': 43, 'z': 44, ',': 45, 'a': 46, '.': 47, 'M': 48, '!': 49, 'O': 50, 'v': 51, "'": 52, 'U': 53, 'w': 54, 'A': 55, 'X': 56, ' ': 57, 'g': 58, 'u': 59, 'F': 60, 'c': 61, 't': 62, 'Y': 63, 'T': 64, 'x': 65, 'k': 66, 'y': 67, '<PAD>': 0, '<S>': 1, '</S>': 2}
68


In [ ]:
SEQ_LENGTH = 200
train_data = data.shuffle(30000).padded_batch(128,
padded_shapes=[SEQ_LENGTH],drop_remainder=True).repeat()


In [ ]:
def split_input_target(chunk):
  input_text = chunk[:,:-1]
  target_text = chunk[:,1:]
  return input_text, target_text

In [ ]:
def split_input_target2(chunk):
  input_text = chunk[:,:-1]
  target_text = chunk[:,1:]
  input_text = tf.one_hot(input_text, depth = vocab_size)
  target_text = tf.one_hot(target_text, depth = vocab_size)
  return input_text, target_text

In [ ]:
train_data = train_data.map(split_input_target)

In [92]:
train_data.take(1)

<TakeDataset shapes: ((128, 199), (128, 199)), types: (tf.int32, tf.int32)>

In [ ]:
train_data2 = train_data.map(split_input_target2)

In [112]:
for input_text, target_text in train_data.take(1):
  print(target_text)
  as_chars = [ind_to_ch[ind] for ind in input_text[3].numpy()]
  print("Input data: "+ "".join(as_chars), "\n")
  as_chars = [ind_to_ch[ind] for ind in target_text[3].numpy()]
  print("Target data: "+ "".join(as_chars), "\n")


tf.Tensor(
[[33  8 40 ...  0  0  0]
 [64 20 55 ...  0  0  0]
 [37 53 33 ...  0  0  0]
 ...
 [ 7 27 42 ...  0  0  0]
 [35 27 40 ...  0  0  0]
 [60 20 50 ...  0  0  0]], shape=(128, 199), dtype=int32)
Input data: <S>LEONTES:
Go on, go on
Thou canst not speak too much; I have deserved
All tongues to talk their bitterest.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD> 

Target data: LEONTES:
Go on, go on
Thou canst not speak too much; I have deserved
All tongues to talk their bitterest.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><P

In [84]:
for input_text2, target_text2 in train_data2.take(1):
  print(input_text2)
  as_chars = [ind_to_ch[ind] for ind in input_text2[3].numpy()]
  print("Input data: "+ "".join(as_chars), "\n")
  as_chars = [ind_to_ch[ind] for ind in target_text2[3].numpy()]
  print("Target data: "+ "".join(as_chars), "\n")

tf.Tensor(
[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]

TypeError: ignored

In [ ]:
# The embedding dimension
embedding_dim = 256

rnn_units = 512
batch_size = 128

In [ ]:
def build_model(vocab_size, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([None, vocab_size], batch_size=batch_size),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = vocab_size,
  rnn_units=rnn_units,
  batch_size=batch_size)

In [145]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (128, None, 512)          1189888   
_________________________________________________________________
dense_3 (Dense)              (128, None, 68)           34884     
Total params: 1,224,772
Trainable params: 1,224,772
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
epochs = 5

In [ ]:

optimizer = tf.optimizers.Adam()


In [ ]:

def train_model(inp, target):
  with tf.GradientTape() as tape:
    mask = tf.cast(tf.not_equal(inp, tf.constant(0)), dtype=tf.float32)
    input_data = tf.one_hot(inp, depth=vocab_size)
    preds = model(input_data)
    remainder = tf.reduce_sum(mask, axis=1)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(target, preds)
    loss = mask * loss
    loss = tf.reduce_sum(loss, axis=1)/ remainder
    loss = tf.reduce_mean(loss)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

input_shape = tf.TensorSpec([None, None], dtype=tf.int32)
train_model = tf.function(train_model, input_signature=(input_shape, input_shape,))

In [ ]:
def loss(labels, logits):
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)

In [167]:
for epoch in range(1):
  for (batch_n, (input_batch, target_batch)) in enumerate(train_data):
    start = time.time()
    model.reset_states()
    loss = train_model(input_batch, target_batch)
    if batch_n % 100 == 0:
      template = 'Batch {} Loss {} Time {}'
      print(template.format(batch_n, loss, time.time()-start))
    if batch_n % 1000 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=batch_n))

Batch 0 Loss 2.9949183464050293 Time 5.158416986465454


KeyboardInterrupt: ignored

In [168]:
checkpoint_dir = './training_checkpoints'
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_0'

In [ ]:
model = build_model(vocab_size, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [170]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, None, 512)            1189888   
_________________________________________________________________
dense_4 (Dense)              (1, None, 68)             34884     
Total params: 1,224,772
Trainable params: 1,224,772
Non-trainable params: 0
_________________________________________________________________
